<h1>Torronto Neighborhood Assignment Notebook 1</h1>

<b>Import Required Libraries</b>

In [18]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#BeautifulSoup for Web Scrapping
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<b>Get complete textual data from given Wikipedia link.</b>

In [19]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

<b>Filter out the required table.</b>

In [20]:
My_table = soup.find('table',{'class':'wikitable sortable'})   #Required Table
My_table = My_table.find('tbody')           #Fetch table body
table_entries=My_table.findAll('td')        #Fetch table rows
len(table_entries)

864

<b>Our tabular data have three columns. categorize and store that data into three respective lists.</b>

In [21]:
neighbourhood =[]
borough = []
postalcode = []
for i in range(2,len(table_entries),3):
    neighbourhood.append(table_entries[i].text.strip())

for i in range(1,len(table_entries),3):
    borough.append(table_entries[i].text.strip())

for i in range(0,len(table_entries),3):
    postalcode.append(table_entries[i].text.strip())
    
print(neighbourhood[:5])
print(borough[:5])
print(postalcode[:5])

['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
['M1A', 'M2A', 'M3A', 'M4A', 'M5A']


<b>Now, organize those lists into a dataframe.</b>

In [22]:
df = pd.DataFrame(columns = ['PostCode','Borough','Neighbourhood'])
df = df.assign(PostCode = postalcode, Borough = borough, Neighbourhood = neighbourhood)
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [23]:
df.shape

(288, 3)

<b>Delete the rows where Borough is Not assigned.</b>

In [24]:
df = df.drop(df[df.Borough == 'Not assigned'].index)
df = df.reset_index(drop=True)
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [25]:
df.shape

(211, 3)

<b>If Neighborhood is Not assigned, then assign 'Borough' value to it.</b>

In [26]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

<b>Group the table according to same 'PostalCode'.</b>
1. Respective Neighborhoods are different. So, they should be joined like A, B, C.
   We simply use join function for this. 
2. Respective Borough names are same. So, no change is required. 
   We apply join function on SETS. Sets cannot have DUPLICATE values.  

In [27]:
df= df.groupby('PostCode', sort = False).agg({'Borough': lambda x: ''.join(set(x)),'Neighbourhood':', '.join}).reset_index(False)
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


<b>Save the Dataframe for later use.</b>

In [30]:
df.to_csv('Toronto_DF1.csv', encoding = 'utf-8', index=False)

<b>Get number of rows and columns.</b>

In [29]:
df.shape

(103, 3)

<b>So, there are 103 rows in our table.</b>